In [5]:
# Load package 


<h1> Exercise 1: The Simplest Consumption Model (Cake Eating Problem)

Consider the simplest consumption-saving model:

$$\begin{aligned}
    V_1^*(W) &= \max_{c_1,c_2,\dots,c_T} \{ \sqrt{c_1} +\beta \sqrt{c_2}+\beta^2\sqrt{c_3}+\cdots+\beta^T\sqrt{c_T} \} \\
    & \text{s.t.} \\
    W &= c_1 + c_1+\cdots+c_T \\
    c_t &\in \mathbb{N}, \, \forall t
\end{aligned}$$
for $T=3$, $\beta =0.9$ and $W=5$.

### 1. Solve the model using backwards induction: 

In [1]:
beta = 0.9
W = 5
T = 3

Vstar_bi = Array{Float64}(undef, W+1,T)
Cstar_bi = Array{Int64}(undef, W+1,T)

Cstar_bi[:,T] = Array(0:W);

Vstar_bi[:,T] = sqrt.(Cstar_bi[:,T] );

In [3]:
Cstar_bi

6×3 Matrix{Int64}:
  1   7  0
  2  -1  1
 -1   9  2
  4  -1  3
  5  11  4
 -1  12  5

In [78]:
Vstar_bi[:,T]

6-element Vector{Float64}:
 0.0
 1.0
 1.4142135623730951
 1.7320508075688772
 2.0
 2.23606797749979

In [81]:
for t = T-1:-1:1
    for w = 0:W
        c = Array(0:w)
        V_next = Vstar_bi[w.-c.+1,t+1]
        v_guess = sqrt.(c)+beta.*V_next    
        
        star = findmax(v_guess)
        #println(v_guess)
        #println(star)
        Vstar_bi[w+1,t] = star[1] 
        Cstar_bi[w+1,t] = star[2]-1
        
        
    end
end

In [82]:
Cstar_bi

6×3 Matrix{Int64}:
 0  0  0
 1  1  1
 1  1  2
 1  2  3
 2  2  4
 2  3  5

### 2. Create a function with $\beta$, $W$ and $T$ as inputs. 

In [83]:
par = Dict("beta" => 0.9,"W" => 5, "T" => 3) 


Dict{String, Real} with 3 entries:
  "W"    => 5
  "T"    => 3
  "beta" => 0.9

In [87]:
par = Dict(beta => 0.9,W => 5, T => 3) 

Dict{Real, Real} with 3 entries:
  5   => 5
  0.9 => 0.9
  3   => 3

In [100]:
struct Par
    beta::Float64
    W::Int64
    T::Int64
end
par = Par(0.9,5,3)

Par(0.9, 5, 3)

In [107]:
function solve_backwards(par)
    
    Vstar_bi = Array{Float64}(undef, par.W+1,par.T)
    Cstar_bi = Array{Int64}(undef, par.W+1,par.T)
    
    # Initialize last period
    Cstar_bi[:,par.T] = Array(0:par.W);
    Vstar_bi[:,par.T] = sqrt.(Cstar_bi[:,par.T] )
    
    # Recursively solve earlier periods: 
    for t = par.T-1:-1:1
        for w = 0:par.W
            c = Array(0:w)
            V_next = Vstar_bi[w.-c.+1,t+1]
            v_guess = sqrt.(c)+par.beta.*V_next    

            star = findmax(v_guess)
            Vstar_bi[w+1,t] = star[1] 
            Cstar_bi[w+1,t] = star[2]-1

        end
    end
    return Cstar_bi,Cstar_bi
end
    

solve_backwards (generic function with 1 method)

### 3. Check that your function works 

Notice, C is our policy function. The policy function tells us the optimal way of spending our resources in a given period. 

In [117]:
par = Par(0.9,5,3)
C, V = solve_backwards(par)
C

6×3 Matrix{Int64}:
 0  0  0
 1  1  1
 1  1  2
 1  2  3
 2  2  4
 2  3  5

In [116]:
par2 = Par(0.9,6,5)
C2, V2 = solve_backwards(par2)
C2

7×5 Matrix{Int64}:
 0  0  0  0  0
 1  1  1  1  1
 1  1  1  1  2
 1  1  1  2  3
 1  1  2  2  4
 1  2  2  3  5
 2  2  2  3  6

### 4. Use the policy function to simulate how 5 resources now can be spend optimally over 3 periods.    

In [131]:
function simulate(Cstar,par)

    C_backwards = Array{Int64}(undef,par.T)
    W_now = par.W
    for t = 1:par.T
        C_now = Cstar[W_now+1,t]
        C_backwards[t] = C_now
        W_now = W_now-C_now  
    end
    return C_backwards
    
end 

simulate (generic function with 2 methods)

In [132]:
C_backwards = simulate(C,par);
C_backwards

3-element Vector{Int64}:
 2
 2
 1

<h3> 5. Solve the model for $T = 10$.

In [133]:
# 1. Setup 

par10 = Par(0.9,5,10)

# 2. - 3. Solve 
C10, V10 = solve_backwards(par10);  # call function

<h3> Simulate $C_1$, $C_2$,...,$C_{10}$ using the policy function found in 5) and W=5

In [134]:
# define function



# 4. Simulate
C_sim = simulate(C10,par10)  # Call function
        
print("Solution for C with T =",T,"and W =", W , ": C =",C_sim)

Solution for C with T =3and W =5: C =[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

<h3> Play around with W, $\beta$ and see the results. Are the results inline with your intuition?

In [136]:
LinRange(0.5,0.99,5)

5-element LinRange{Float64}:
 0.5,0.6225,0.745,0.8675,0.99

In [145]:
for beta in LinRange(0.5,0.99,5)
    println("Beta: ",beta)
    
    for W = 5:3:15
        println("  W: ", W)
        parnow = Par(beta,W,5)
        C_now, V_now = solve_backwards(parnow)
        C_sim = simulate(C_now,parnow)
        
        println("    V: ",V_now[W+1,1])
        println("    C: ",C_sim)
    end
end 
        
        
        

Beta: 0.5
  W: 5
    V: 4
    C: [4, 1, 0, 0, 0]
  W: 8
    V: 6
    C: [6, 1, 1, 0, 0]
  W: 11
    V: 8
    C: [8, 2, 1, 0, 0]
  W: 14
    V: 10
    C: [10, 3, 1, 0, 0]
Beta: 0.6225
  W: 5
    V: 3
    C: [3, 1, 1, 0, 0]
  W: 8
    V: 4
    C: [4, 2, 1, 1, 0]
  W: 11
    V: 6
    C: [6, 3, 1, 1, 0]
  W: 14
    V: 9
    C: [9, 3, 1, 1, 0]
Beta: 0.745
  W: 5
    V: 2
    C: [2, 1, 1, 1, 0]
  W: 8
    V: 3
    C: [3, 2, 1, 1, 1]
  W: 11
    V: 5
    C: [5, 3, 1, 1, 1]
  W: 14
    V: 6
    C: [6, 4, 2, 1, 1]
Beta: 0.8674999999999999
  W: 5
    V: 1
    C: [1, 1, 1, 1, 1]
  W: 8
    V: 3
    C: [3, 2, 1, 1, 1]
  W: 11
    V: 3
    C: [3, 3, 2, 2, 1]
  W: 14
    V: 5
    C: [5, 3, 3, 2, 1]
Beta: 0.99
  W: 5
    V: 1
    C: [1, 1, 1, 1, 1]
  W: 8
    V: 2
    C: [2, 2, 2, 1, 1]
  W: 11
    V: 3
    C: [3, 2, 2, 2, 2]
  W: 14
    V: 3
    C: [3, 3, 3, 3, 2]
